In [1]:
import sys 
sys.path.append("C:/Users/L2S/Desktop/Efficient_and_reliable_GP/")

import gpmp.num as gnp
from gpmp.kernel import maternp_kernel

from Code.data_generation   .samplers                      import sampler, extract_K_samples
from Code.gaussian_processes.gaussian_process              import GaussianProcessModel
from Code.Parameters        .sampler_parameters            import sampler_parameters
from Code.Parameters        .gaussian_process_parameters   import gaussian_process_model_parameters

Using backend: torch


### Dataset generation

In [2]:
N_function             = 500
N_train                = 25
function_name          = "Power sinus 1D"
function_X, function_Y = sampler          (N_function, function_name, sampler_parameters)
train_X   , train_Y    = extract_K_samples(function_X, function_Y, N_train)

# 1 - Gaussian process regression

In [3]:
gaussian_process_model                 = GaussianProcessModel                                      (train_X, train_Y, gaussian_process_model_parameters)
gaussian_process_model.train                                                                       ()
hyperparameters                        = gaussian_process_model.get_hyperparameters                () 
covparam                               = gaussian_process_model.get_gaussian_process_model_covparam()
optimization_infos                     = gaussian_process_model.optimization_infos                 
y_posterior_mean, y_posterior_variance = gaussian_process_model.predict                            (function_X)

In [4]:
print(hyperparameters, type(hyperparameters))

{'Time': 0.09233403205871582, 'p': 5, 'mu': -0.13523242255667978, 'sigma': 0.8340344289608462, 'Length scales': [0.6180975975741381], 'Nugget': 1e-08} <class 'dict'>


In [5]:
print(covparam, type(covparam))

tensor([-0.3630,  0.4811]) <class 'torch.Tensor'>


In [6]:
print(optimization_infos, type(optimization_infos))

{'Hyperparameters history': [array([-1.30276673,  0.81879683]), array([ -1.10382217, -25.10699022]), array([-1.28287227, -1.77378187]), array([-1.30077728,  0.55953896]), array([ 4.48432768, -0.54405944]), array([-0.44897258,  0.39704402]), array([-0.39723342,  0.55987833]), array([-0.38870119,  0.50486035]), array([-0.36869435,  0.47794894]), array([-0.36639049,  0.48186936]), array([-0.36412583,  0.48132677]), array([-0.36296119,  0.48110891])], 'Criterion history': [-121.09840511584753, 2423261637.071649, 16095175.661505431, -121.93024380918328, -31.99908078150075, -124.42054892136841, -125.34019375187236, -125.88230038114781, -125.92683240312178, -125.92906301980591, -125.9290928647304, -125.92909636989512], 'Number of training iterations': 12} <class 'dict'>


In [7]:
print(y_posterior_mean[:5], type(y_posterior_mean[:5]))

[0.00493132 0.0183627  0.02486698 0.02635244 0.04485918] <class 'numpy.ndarray'>


In [8]:
print(y_posterior_variance[:5], type(y_posterior_variance[:5]))

[9.94455815e-05 8.96273760e-05 8.51713621e-05 8.41800025e-05
 7.26119657e-05] <class 'numpy.ndarray'>


# 2 - Set custom hyperparameters

In [9]:
sigma2           = 1
rho              = [1]
gaussian_process_model.set_hyperparameters(sigma2, rho)
hyperparameters  = gaussian_process_model.get_hyperparameters()
print(hyperparameters)

{'Time': 0.09233403205871582, 'p': 5, 'mu': -0.13523242255667978, 'sigma': 1.0, 'Length scales': [1.0], 'Nugget': 1e-08}


# 3 - Covariance matrix computation

In [10]:
covparam  = gaussian_process_model._gaussian_process_model.covparam
sigma2    = gnp.exp(covparam[0])
loginvrho = covparam[1:]

In [11]:
Sigma = gaussian_process_model.compute_covariance_matrix(train_X, train_X)
Sigma

tensor([[1.0000, 0.9631, 0.9090, 0.9314, 0.7971, 0.9342, 0.9999, 0.8993, 0.9594,
         0.8609, 0.8524, 0.8628, 0.8860, 0.9543, 0.9384, 0.8697, 0.9719, 0.8903,
         0.9264, 0.7858, 0.7534, 0.8330, 0.9233, 0.8565, 0.9287],
        [0.9631, 1.0000, 0.9866, 0.9946, 0.9208, 0.9955, 0.9666, 0.7682, 0.9999,
         0.7201, 0.7099, 0.7224, 0.9760, 0.8473, 0.9966, 0.9673, 0.8782, 0.9781,
         0.9931, 0.9126, 0.8879, 0.6875, 0.8006, 0.9598, 0.8084],
        [0.9090, 0.9866, 1.0000, 0.9982, 0.9708, 0.9976, 0.9143, 0.6782, 0.9887,
         0.6285, 0.6183, 0.6309, 0.9984, 0.7641, 0.9967, 0.9956, 0.7996, 0.9989,
         0.9989, 0.9654, 0.9483, 0.5957, 0.7127, 0.9925, 0.7211],
        [0.9314, 0.9946, 0.9982, 1.0000, 0.9549, 1.0000, 0.9361, 0.7119, 0.9960,
         0.6625, 0.6522, 0.6648, 0.9932, 0.7960, 0.9998, 0.9882, 0.8301, 0.9943,
         0.9999, 0.9484, 0.9282, 0.6296, 0.7459, 0.9833, 0.7541],
        [0.7971, 0.9208, 0.9708, 0.9549, 1.0000, 0.9525, 0.8042, 0.5431, 0.9258,
       

In [12]:
p      = gaussian_process_model_parameters["Matern parameter"]
nugget = gaussian_process_model_parameters["Nugget"          ]
K      = gnp.scaled_distance(loginvrho, train_X, train_X) 
K      = sigma2 * maternp_kernel(p, K)
K      = K + nugget * gnp.eye(K.shape[0])
K

tensor([[1.0000, 0.9631, 0.9090, 0.9314, 0.7971, 0.9342, 0.9999, 0.8993, 0.9594,
         0.8609, 0.8524, 0.8628, 0.8860, 0.9543, 0.9384, 0.8697, 0.9719, 0.8903,
         0.9264, 0.7858, 0.7534, 0.8330, 0.9233, 0.8565, 0.9287],
        [0.9631, 1.0000, 0.9866, 0.9946, 0.9208, 0.9955, 0.9666, 0.7682, 0.9999,
         0.7201, 0.7099, 0.7224, 0.9760, 0.8473, 0.9966, 0.9673, 0.8782, 0.9781,
         0.9931, 0.9126, 0.8879, 0.6875, 0.8006, 0.9598, 0.8084],
        [0.9090, 0.9866, 1.0000, 0.9982, 0.9708, 0.9976, 0.9143, 0.6782, 0.9887,
         0.6285, 0.6183, 0.6309, 0.9984, 0.7641, 0.9967, 0.9956, 0.7996, 0.9989,
         0.9989, 0.9654, 0.9483, 0.5957, 0.7127, 0.9925, 0.7211],
        [0.9314, 0.9946, 0.9982, 1.0000, 0.9549, 1.0000, 0.9361, 0.7119, 0.9960,
         0.6625, 0.6522, 0.6648, 0.9932, 0.7960, 0.9998, 0.9882, 0.8301, 0.9943,
         0.9999, 0.9484, 0.9282, 0.6296, 0.7459, 0.9833, 0.7541],
        [0.7971, 0.9208, 0.9708, 0.9549, 1.0000, 0.9525, 0.8042, 0.5431, 0.9258,
       